# Build Substack Text Database

The goal of this is to build a database that we can use to identify interesting keywords, topics, writing patterns, and themes, etc. that are associated with higher levels of subscribers and engagement for writers. 

### Load Libraries, Setting Up Database

In [1]:
# Import Libraries
import requests
import pandas as pd
import numpy as np
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import json
import re
import traceback
import sys
import sqlite3

In [25]:
db_file = "substack_database.db"
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Create the blog_metadata table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS blog_metadata (
        blog_url TEXT PRIMARY KEY,
        subscriber_count INTEGER,
        public_post_count INTEGER,
        private_post_count INTEGER
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS article_data (
        title TEXT,
        audience TEXT,
        canonical_url TEXT PRIMARY KEY,
        description TEXT,
        truncated_body_text TEXT,
        wordcount INTEGER,
        reaction_count INTEGER,
        comment_count INTEGER,
        post_date TEXT, 
        polarity REAL,
        objectivity REAL,
        number_of_questions INTEGER,
        fk_grade_level REAL,
        gunning_fog_index REAL,
        reading_time REAL,
        p_elem_counts INTEGER,
        a_elem_counts INTEGER,
        img_elem_counts INTEGER,
        ul_elem_counts INTEGER,
        li_elem_counts INTEGER,
        video_elem_counts INTEGER,
        br_elem_counts INTEGER,
        tokens TEXT
    )
''')


# Commit changes and close the database connection
conn.commit()
conn.close()

### Identify Substack Blogs For Scraping
Here we identify substack blogs and get the number of subscribers per blog

In [ ]:
# Set the query and the desired website domain to search
# it would be nice to have a better way to do this.... 
query = "site:substack.com"
num_results = 1000

# Create a list to store the URLs
results = []

# Perform the Google search and scrape the URLs
for url in tqdm(search(query, num_results=num_results)):
    results.append(url)
    #time.sleep(1) # space out requests so we dont get blocked by google


In [ ]:
results

In [ ]:
# extract only blogs that are substack sites
results = [x for x in results if ".substack.com" in x]

# remove any sites that are blogs, we only want the original homepage
filtered_results = []
for url in results:
    if "/p/" in url:
        filtered_results.append(url.split("/p/")[0])
    else:
        filtered_results.append(url)

# remove duplicates
filtered_results = list(set(filtered_results))
filtered_results

In [ ]:
# get the number of subscribers for each blog

subscriber_counts = []
for url in tqdm(filtered_results):
    # get subscriber count for the blog
    subscribers_count = None
    script_element = None
    json_data = None
    parsed_dict = None
    formatted_dict = None
    subscribers_count_str = None

    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text)
        # Assuming you have the HTML content of the page loaded into BeautifulSoup as 'soup'
        # Find the script element containing the JSON data
        script_elements = soup.find_all("script")

        for script_element in script_elements:
            if "subscribers" in script_element.text:
                break

        input_string = script_element.text
        
        # Find the JSON data within the string
        start_index = input_string.find('JSON.parse(') + len('JSON.parse(')
        end_index = input_string.rfind(');', start_index)

        # Extract the JSON data
        json_data = input_string[start_index:end_index].strip()

        # Parse the JSON data into a dictionary
        parsed_dict = json.loads(json_data)

        def extract_value_by_key(dictionary, target_key):
            # Check if the target_key is in the current dictionary
            if target_key in dictionary:
                return dictionary[target_key]
            
            # If the key is not found, recursively search in nested dictionaries
            for key, value in dictionary.items():
                if isinstance(value, dict):
                    result = extract_value_by_key(value, target_key)
                    if result is not None:
                        return result
            
            # If the key is not found anywhere in the dictionary, return None
            return None

        formatted_dict = json.loads(parsed_dict)

        target_key = "rankingDetailFreeSubscriberCount"
        subscriber_str = extract_value_by_key(formatted_dict, target_key)

        # Use regular expressions to find the number with commas in the string
        match = re.search(r'\d{1,3}(?:,\d{3})*(?:\.\d+)?', subscriber_str)

        if match:
            subscribers_count_str = match.group()
            # Remove commas and convert to an integer
            subscribers_count = int(subscribers_count_str.replace(',', ''))
        else:
            print("No subscribers count found in the string.")
            subscribers_count = None
    except Exception as e:
        print("Error", e)
        subscribers_count = None
        # Print the traceback
        exc_type, exc_value, exc_traceback = sys.exc_info()
        traceback.print_tb(exc_traceback)

    subscriber_counts.append(subscribers_count)
    time.sleep(5)

In [17]:
# assemble metadata
metadata_df = pd.DataFrame()
metadata_df['blog'] = filtered_results
metadata_df['subscribers'] = subscriber_counts
metadata_df

,blog,subscribers
0,https://krystalkyleandfriends.substack.com/,44000.0
1,https://chamath.substack.com,69000.0
2,https://chrishedges.substack.com/,72000.0
3,https://gingerriver.substack.com/,5000.0
4,https://jessicadefino.substack.com,90000.0
...,...,...
379,https://anneboyer.substack.com/,7000.0
380,https://alicebell.substack.com/,9000.0
381,https://jeffreycarr.substack.com/,3000.0
382,https://nathanbenaich.substack.com/,24000.0


In [18]:
# remove blogs where we could not collect subscriber counts
metadata_df = metadata_df.dropna()
metadata_df

,blog,subscribers
0,https://krystalkyleandfriends.substack.com/,44000.0
1,https://chamath.substack.com,69000.0
2,https://chrishedges.substack.com/,72000.0
3,https://gingerriver.substack.com/,5000.0
4,https://jessicadefino.substack.com,90000.0
...,...,...
379,https://anneboyer.substack.com/,7000.0
380,https://alicebell.substack.com/,9000.0
381,https://jeffreycarr.substack.com/,3000.0
382,https://nathanbenaich.substack.com/,24000.0


In [23]:
# Connect to the SQLite database and save the data
db_file = "substack_database.db"
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Iterate through the DataFrame and insert rows into blog_metadata
for index, row in metadata_df.iterrows():
    blog_url = row['blog']
    subscriber_count = row['subscribers']

    # Check if the blog_url already exists in the table
    cursor.execute("SELECT blog_url FROM blog_metadata WHERE blog_url=?", (blog_url,))
    existing_row = cursor.fetchone()

    if not existing_row:
        # Insert the row into the table if it doesn't exist
        cursor.execute("INSERT INTO blog_metadata (blog_url, subscriber_count) VALUES (?, ?)",
                       (blog_url, subscriber_count))

# Commit changes and close the database connection
conn.commit()
conn.close()

### Identify Articles From Substack Blogs

In [26]:
# collect all article data for all blogs that we have available.
from utils import get_posts_for_blog, get_post_metadata

for x in tqdm(range(0, len(metadata_df))):
    try:
        blog_url = metadata_df.iloc[x]['blog'].replace(".com/", ".com")
        blog_subscribers = metadata_df.iloc[x]['subscribers'] 


        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # Retrieve all canonical_urls from the article_data table - need to check if we've already collected articles for this blog. 
        cursor.execute("SELECT canonical_url FROM article_data")
        results = cursor.fetchall()

        # Close the database connection
        conn.close()

        # Extract the first part of each URL and add ".com" back to it
        unique_domains = set()
        for result in results:
            url = result[0]
            domain_parts = url.split('.com')
            if len(domain_parts) > 0:
                unique_domains.append(domain_parts[0] + ".com")

        # if the blog has already been connected, dont collect the data it
        if blog_url in unique_domains:
            pass
        else:
            post_data = get_posts_for_blog(blog_url)
            
            conn = sqlite3.connect(db_file)
            cursor = conn.cursor()

            # Iterate through the DataFrame and insert rows into article_data
            for index, row in post_data.iterrows():
                title = row['title']
                audience = row['audience']
                canonical_url = row['canonical_url']
                description = row['description']
                truncated_body_text = row['truncated_body_text']
                wordcount = row['wordcount']
                reaction_count = row['reaction_count']
                comment_count = row['comment_count']
                post_date = row['post_date']

                # Check if the canonical_url already exists in the table
                cursor.execute("SELECT canonical_url FROM article_data WHERE canonical_url=?", (canonical_url,))
                existing_row = cursor.fetchone()

                if not existing_row:
                    # Insert the row into the table if it doesn't exist
                    cursor.execute("INSERT INTO article_data (title, audience, canonical_url, description, truncated_body_text, wordcount, reaction_count, comment_count, post_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
                                (title, audience, canonical_url, description, truncated_body_text, wordcount, reaction_count, comment_count, post_date))

            # Commit changes and close the database connection
            conn.commit()
            conn.close()

    except:
        pass



 13%|█▎        | 39/309 [1:18:03<6:08:21, 81.86s/it]  

### Download Articles, Calculate Metrics
After identifying article links, we still need to download the article text and calculate some metrics we'll use later for each article.

In [17]:
# add more metadata to all article data
from utils import get_post_metadata_from_url

def insert_results_into_db(db_path, url, results):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Convert the 'tokens_results' dictionary to a JSON string
    tokens_results_json = json.dumps(results["tokens_results"])

    # Update the database row with the obtained results
    cursor.execute('''
        UPDATE article_data
        SET
            p_elem_counts = ?,
            img_elem_counts = ?,
            a_elem_counts = ?,
            ul_elem_counts = ?,
            li_elem_counts = ?,
            video_elem_counts = ?,
            br_elem_counts = ?,
            tokens = ?,
            polarity = ?,
            objectivity = ?,
            number_of_questions = ?,
            fk_grade_level = ?,
            gunning_fog_index = ?,
            reading_time = ?
        WHERE canonical_url = ?
    ''', (
        results["p_elem_counts"],
        results["img_elem_counts"],
        results["a_elem_counts"],
        results["ul_elem_counts"],
        results["li_elem_counts"],
        results["video_elem_counts"],
        results["br_elem_counts"],
        tokens_results_json,
        results["polarity_results"],
        results["objectivity_results"],
        results["num_questions_results"],
        results["fk_grade_level_results"],
        results["gunning_fog_index_results"],
        results["reading_time_results"],
        url
    ))

    # Commit the changes and close the database connection
    conn.commit()
    conn.close()

# first, get all rows where metadata hasnt been collected (all the fields are null)
db_file = "substack_database.db"
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

cursor.execute('''
    SELECT canonical_url
    FROM article_data
    WHERE polarity IS NULL
      AND objectivity IS NULL
      AND number_of_questions IS NULL
      AND fk_grade_level IS NULL
      AND gunning_fog_index IS NULL
      AND reading_time IS NULL
      AND p_elem_counts IS NULL
      AND a_elem_counts IS NULL
      AND img_elem_counts IS NULL
      AND ul_elem_counts IS NULL
      AND li_elem_counts IS NULL
      AND video_elem_counts IS NULL
      AND br_elem_counts IS NULL
      AND tokens IS NULL
      AND audience = "everyone"
''')

# Fetch all the rows that match the criteria
rows = cursor.fetchall()
# Close the database connection
conn.close()

for row in tqdm(rows):
    try:
        url = row[0]
        results = get_post_metadata_from_url(url)
        insert_results_into_db(db_file, url, results)
    except:
        pass
    

  0%|          | 0/6547 [00:00<?, ?it/s]/Users/hansenhan/substack_scraper/utils.py:289: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 289 of the file /Users/hansenhan/substack_scraper/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  r = requests.get(post_url)
/Users/hansenhan/substack_scraper/utils.py:297: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused thi